In [1]:
from pathlib import Path
import open3d as o3d
import os

from pytorch_lightning import seed_everything

from src.dataset_utils import (
    get_singleview_data,
    get_multiview_data,
    get_voxel_data_json,
    get_image_transform_latent_model,
    get_pointcloud_data,
    get_mv_dm_data,
    get_sv_dm_data,
    get_sketch_data
)
from src.model_utils import Model
from src.mvdream_utils import load_mvdream_model
import argparse
from PIL import Image


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1518790706.py, line 1)

In [2]:
from comet_ml import start
from comet_ml.integration.pytorch import log_model

experiment = start(
  api_key="mqrUAXjKBRul24uX6pxR3gRHX*eyJiYXNlVXJsIjoiaHR0cHM6Ly9jb21ldC5kZXYuY2xvdWRvcy5hdXRvZGVzay5jb20ifQ",
  project_name="wala-time-checks",
  workspace="alessandro-giuliano"
)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://comet.dev.cloudos.autodesk.com/alessandro-giuliano/wala-time-checks/c05636669e7f48468c3f981dc63b9a77



In [3]:

def simplify_mesh(obj_path, target_num_faces=1000):
    mesh = o3d.io.read_triangle_mesh(obj_path)
    simplified_mesh = mesh.simplify_quadric_decimation(target_num_faces)
    o3d.io.write_triangle_mesh(obj_path, simplified_mesh)


def generate_3d_object(
    model,
    data,
    data_idx,
    scale,
    diffusion_rescale_timestep,
    save_dir="examples",
    output_format="obj",
    target_num_faces=None,
    seed=42,
):
    # Set seed
    seed_everything(seed, workers=True)

    save_dir.mkdir(parents=True, exist_ok=True)
    model.set_inference_fusion_params(scale, diffusion_rescale_timestep)
    output_path = model.test_inference(
        data, data_idx, save_dir=save_dir, output_format=output_format
    )

    if output_format == "obj" and target_num_faces:
        simplify_mesh(output_path, target_num_faces=target_num_faces)


In [ ]:
model_name = 'ADSKAILab/WaLa-SV-1B'
images_path = Path('examples/single_view/')
single_image = ['examples/single_view/table.png']

output_dir = 'examples/Test_Gen'
output_format = 'obj'
target_num_faces = None
scale = 1.8
seed = 42
diffusion_rescale_timestep = 5



In [5]:
print(f"Loading model")

model = Model.from_pretrained(pretrained_model_name_or_path=model_name)
image_transform = get_image_transform_latent_model()

for image_path in single_image:
    print(f"Processing image: {image_path}")
    data = get_singleview_data(
        image_file=Path(image_path),
        image_transform=image_transform,
        device=model.device,
        image_over_white=False,
    )
    data_idx = 0
    save_dir = Path(output_dir) / Path(image_path).stem

    model.set_inference_fusion_params(
        scale, diffusion_rescale_timestep
    )

    generate_3d_object(
        model,
        data,
        data_idx,
        scale,
        diffusion_rescale_timestep,
        save_dir,
        output_format,
        target_num_faces,
        seed,
    )


    

Loading model
'DotDict' object has no attribute 'dataset_path'
'DotDict' object has no attribute 'low_avg'
'DotDict' object has no attribute 'low_avg'
Low avg used : None high value: 63


/opt/miniconda/envs/wala/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.3.3, which is newer than your current Lightning version: v2.1.0
Using cache found in /home/ray/.cache/torch/hub/facebookresearch_dinov2_main
/home/ray/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/ray/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/ray/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


cond_emb_dim: 1024
Input resolution: 224
Vocab size: N/A
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


Seed set to 42


Processing image: examples/single_view/table.png
Extract Image 0.12082266807556152 s
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


  0%|          | 0/5 [00:00<?, ?it/s]

Latent Diffusion Time 2.3993334770202637 s
Latent Decoding Time 0.08286499977111816 s
Wavelet Preparation Time 0.0002429485321044922 s
Low to Highs conversion 0.2597346305847168 s
Inverse DWT time elapsed 0.007523298263549805 s
mcubes.marching_cubes time 0.5941355228424072 s
export obj time 2.3561811447143555 s
Time to save visualization from high lows to obj 2.9584014415740967 s


In [ ]:
import time


for image_path in images_path.iterdir():
    timer1 = time.time()
    print(f"Processing image: {image_path}")
    data = get_singleview_data(
        image_file=Path(image_path),
        image_transform=image_transform,
        device=model.device,
        image_over_white=False,
    )
    data_idx = 0
    save_dir = Path(output_dir) 

    model.set_inference_fusion_params(
        scale, diffusion_rescale_timestep
    )

    generate_3d_object(
        model,
        data,
        data_idx,
        scale,
        diffusion_rescale_timestep,
        save_dir,
        output_format,
        target_num_faces,
        seed,
    )
    delta = time.time() - timer1
    experiment.log_metric("Default Delta", delta)
    print('Total Inference time', delta)

experiment.end()

Seed set to 42


Processing image: examples/single_view/table.png
Extract Image 0.015177249908447266 s
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


  0%|          | 0/5 [00:00<?, ?it/s]

Latent Diffusion Time 2.3518030643463135 s
Latent Decoding Time 0.00409698486328125 s
Wavelet Preparation Time 5.1021575927734375e-05 s
Low to Highs conversion 0.05619668960571289 s
Inverse DWT time elapsed 0.0030546188354492188 s
mcubes.marching_cubes time 0.5514864921569824 s


Seed set to 42


...

Latent Diffusion Time 2.354640245437622 s
Latent Decoding Time 0.0042340755462646484 s
Wavelet Preparation Time 5.0067901611328125e-05 s
Low to Highs conversion 0.056107521057128906 s
Inverse DWT time elapsed 0.003101348876953125 s
mcubes.marching_cubes time 0.5492887496948242 s


Seed set to 42


export obj time 1.993685007095337 s
Time to save visualization from high lows to obj 2.54648756980896 s
Total Inference time 4.983244180679321
Processing image: examples/single_view/pear.jpeg
Extract Image 0.014913558959960938 s
'DotDict' object has no attribute 'use_multiple_views_inferences'
'DotDict' object has no attribute 'use_multiple_views_inferences'


  0%|          | 0/5 [00:00<?, ?it/s]

Latent Diffusion Time 2.3528528213500977 s
Latent Decoding Time 0.004122734069824219 s
Wavelet Preparation Time 4.9114227294921875e-05 s
Low to Highs conversion 0.05606794357299805 s
Inverse DWT time elapsed 0.0033533573150634766 s
mcubes.marching_cubes time 0.5497264862060547 s
export obj time 1.7510895729064941 s
Time to save visualization from high lows to obj 2.3045737743377686 s
Total Inference time 4.740160703659058


In [ ]:
####### Dataset download and extraction

In [9]:
import zipfile
from pathlib import Path
import os
import shutil

zip_dir = '/home/ray/WaLa/Raw_data'         # Folder with zip files
output_dir = '/home/ray/WaLa/test_images'  # Where to save extracted thumbnails
os.makedirs(output_dir, exist_ok=True)

for zip_path in Path(zip_dir).glob('*.zip'):
    with zipfile.ZipFile(zip_path, 'r') as z:
        # Find files in the 'thumbnail' folder (not subfolders)
        thumbnail_files = [f for f in z.namelist() if f.startswith('thumbnails/') and not f.endswith('/')]
        if thumbnail_files:
            thumb_file = thumbnail_files[0]  # Pick the first one
            ext = Path(thumb_file).suffix
            out_name = zip_path.stem + ext
            out_path = Path(output_dir) / out_name
            with z.open(thumb_file) as source, open(out_path, 'wb') as target:
                shutil.copyfileobj(source, target)
            print(f"Extracted {thumb_file} from {zip_path.name} as {out_name}")

Extracted thumbnails/0.jpg from 2_of_Jenga_Classic_Game.zip as 2_of_Jenga_Classic_Game.jpg
Extracted thumbnails/0.jpg from 50_BLOCKS.zip as 50_BLOCKS.jpg
Extracted thumbnails/0.jpg from 5_HTP.zip as 5_HTP.jpg
Extracted thumbnails/0.jpg from ASICS_GELAce_Pro_Pearl_WhitePink.zip as ASICS_GELAce_Pro_Pearl_WhitePink.jpg
Extracted thumbnails/0.jpg from ASICS_GELBlur33_20_GS_BlackWhiteSafety_Orange.zip as ASICS_GELBlur33_20_GS_BlackWhiteSafety_Orange.jpg
Extracted thumbnails/0.jpg from Asus_M5A78LMUSB3_Motherboard_Micro_ATX_Socket_AM3.zip as Asus_M5A78LMUSB3_Motherboard_Micro_ATX_Socket_AM3.jpg
Extracted thumbnails/0.jpg from Asus_M5A99FX_PRO_R20_Motherboard_ATX_Socket_AM3.zip as Asus_M5A99FX_PRO_R20_Motherboard_ATX_Socket_AM3.jpg
Extracted thumbnails/0.jpg from Asus_Sabertooth_990FX_20_Motherboard_ATX_Socket_AM3.zip as Asus_Sabertooth_990FX_20_Motherboard_ATX_Socket_AM3.jpg
Extracted thumbnails/0.jpg from Asus_Sabertooth_Z97_MARK_1_Motherboard_ATX_LGA1150_Socket.zip as Asus_Sabertooth_Z97_M

BadZipFile: File is not a zip file

In [12]:
from pathlib import Path

def count_extracted_images(target_folder, extensions={'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}):
    """
    Counts the number of image files in the target folder.
    Args:
        target_folder (str or Path): Path to the folder to check.
        extensions (set): Set of file extensions to consider as images.
    Returns:
        int: Number of image files found.
    """
    target_folder = Path(target_folder)
    return sum(1 for f in target_folder.iterdir() if f.suffix.lower() in extensions and f.is_file())

num_images = count_extracted_images('/home/ray/WaLa/test_images')  # Where to save extracted thumbnails
print(f"Extracted images: {num_images}")

Extracted images: 999
